<a href="https://colab.research.google.com/github/carlosfmorenog/CM4125/blob/main/CM4125_T7/CM4125_Lec7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CM4125 Topic 7: Data Manipulation (Part 2)

##  Joins and Aggregations

Let's create two small dataset to use as example

In [ ]:
import pandas as pd
books = pd.DataFrame({'Author' : ['J. R. R. Tolkien',
                                  'George R. R. Martin',
                                  'J. K. Rowling',
                                  'Suzanne Collins']},
                     index = ['The Lord of the Rings',
                              'Game of Thrones',
                              'Harry Potter',
                              'The Hunger Games'])
books

,Author
The Lord of the Rings,J. R. R. Tolkien
Game of Thrones,George R. R. Martin
Harry Potter,J. K. Rowling
The Hunger Games,Suzanne Collins


In [ ]:
films = pd.DataFrame({'Year of First Film' : [1999, 2001, 2001, 2012],
                      'Number of Films' : [3, 2, 8, 4]},
                     index = ['The Matrix',
                              'The Lord of the Rings',
                              'Harry Potter',
                              'The Hunger Games'])
films

,Year of First Film,Number of Films
The Matrix,1999,3
The Lord of the Rings,2001,2
Harry Potter,2001,8
The Hunger Games,2012,4


The `.join` operation is used to join the columns of two different datasets based on matching index

In [ ]:
books.join(films)

,Author,Year of First Film,Number of Films
The Lord of the Rings,J. R. R. Tolkien,2001.0,2.0
Game of Thrones,George R. R. Martin,NaN,NaN
Harry Potter,J. K. Rowling,2001.0,8.0
The Hunger Games,Suzanne Collins,2012.0,4.0


### Types of Joins

In the previous example, the `books` dataset is the *'left'* dataset and `films` is the *'right'* dataset

A left join keeps all of the data from the *'left'* dataset and adds in the applicable data from the *'right'* dataset where the keys match up

If there is no film, the cells are populated with `NaN` (e.g. Game of Thrones)

The above operations is equivalent to

    books.join(films, how='left')

We can also do a right join. This keeps all of the films, adding the book data where applicable. If there is no book, the cells are populated with `NaN`.

In [ ]:
books.join(films, how='right')

,Author,Year of First Film,Number of Films
The Matrix,NaN,1999,3
The Lord of the Rings,J. R. R. Tolkien,2001,2
Harry Potter,J. K. Rowling,2001,8
The Hunger Games,Suzanne Collins,2012,4


This is almost equivalent to `films.join(books)` (or `films.join(books, how='left')`), with the exception of the order in which the columns appear

If we want to keep all of the data (book **OR** film), we can use an outer join

In [ ]:
films.join(books, how='outer')

,Year of First Film,Number of Films,Author
Game of Thrones,NaN,NaN,George R. R. Martin
Harry Potter,2001.0,8.0,J. K. Rowling
The Hunger Games,2012.0,4.0,Suzanne Collins
The Lord of the Rings,2001.0,2.0,J. R. R. Tolkien
The Matrix,1999.0,3.0,NaN


And if we only want to keep the data (book **AND** film), we can use an inner join

In [ ]:
films.join(books, how='inner')

,Year of First Film,Number of Films,Author
The Lord of the Rings,2001,2,J. R. R. Tolkien
Harry Potter,2001,8,J. K. Rowling
The Hunger Games,2012,4,Suzanne Collins


You should choose the join type based on what your resulting data table is intended to describe

For instance, the inner join gave us a table of films based on books

Contrarily, the left join gave us a list of books with additional information on the film (if any)

Once again, remember that `.join` is not modifying the dataframe, so if you want to save the result, assign it to either the same or a different variable with an appropriate name

    films = films.join(books, how='left')

    books = films.join(books, how='right')
    
    films_on_books = films.join(books, how='inner')
    
    favourite_series = films.join(books, how='outer')

This table may be used as a reminder of the difference between the joins


| Type of Join   | Keeps Rows of Left Data | Keeps Rows of Right Data |
| :------------- | ----------------------: | -----------------------: |
| left (default) | yes                     | only if matching left    |
| right          | only if matching right  | yes                      |
| outer          | yes                     | yes                      |
| inner          | only if matching right  | only if matching left    |

### Joining Different Columns

`.join` joins by comparing indexes of each dataframe

Sometimes the key column(s) is not the index (particularly if you are using default indexing)

If you need to join based on columns other than the index, you should use `merge`

For example, it is possible that we would encounter data with default indexes as follows:

In [ ]:
books = books.reset_index()
books = books.rename(columns={'index' : 'Book Series Title'})
books

,Book Series Title,Author
0,The Lord of the Rings,J. R. R. Tolkien
1,Game of Thrones,George R. R. Martin
2,Harry Potter,J. K. Rowling
3,The Hunger Games,Suzanne Collins


In [ ]:
films = films.reset_index()
films = films.rename(columns={'index' : 'Film Series Title'})
films

,Film Series Title,Year of First Film,Number of Films
0,The Matrix,1999,3
1,The Lord of the Rings,2001,2
2,Harry Potter,2001,8
3,The Hunger Games,2012,4


If we join on the index, the result is nonsense!

In [ ]:
books.join(films) # WRONG

,Book Series Title,Author,Film Series Title,Year of First Film,Number of Films
0,The Lord of the Rings,J. R. R. Tolkien,The Matrix,1999,3
1,Game of Thrones,George R. R. Martin,The Lord of the Rings,2001,2
2,Harry Potter,J. K. Rowling,Harry Potter,2001,8
3,The Hunger Games,Suzanne Collins,The Hunger Games,2012,4


We could change the `Book Series Title` and `Film Series Title` to indexes and join with `.join`

Or we can use `.merge`, in which left, right, outer, and inner joins work the same way

However, it is not the index we are comparing, it is the column specified with `left_on=` and `right_on=`

In [ ]:
books.merge(films,
            how='inner',
            left_on='Book Series Title',
            right_on='Film Series Title')

,Book Series Title,Author,Film Series Title,Year of First Film,Number of Films
0,The Lord of the Rings,J. R. R. Tolkien,The Lord of the Rings,2001,2
1,Harry Potter,J. K. Rowling,Harry Potter,2001,8
2,The Hunger Games,Suzanne Collins,The Hunger Games,2012,4


### Data Aggregation

Before continuing, let's reset the books index

In [ ]:
books = books.set_index('Book Series Title', drop=True)
books

,Author
Book Series Title,
The Lord of the Rings,J. R. R. Tolkien
Game of Thrones,George R. R. Martin
Harry Potter,J. K. Rowling
The Hunger Games,Suzanne Collins


Let's import a dataset of a list of books

In [ ]:
volumes = pd.read_csv(
    'https://www.dropbox.com/scl/fi/q97rwhegcreaiy3cm0pq7/volumes.csv?rlkey=shl0fmr32gaz8fyofnwbrg043&raw=1')
volumes

,Series,Title,Rating,Year
0,Harry Potter,Harry Potter and the Philosopher's Stone,4.47,1997
1,Harry Potter,Harry Potter and the Chamber of Secrets,4.42,1998
2,Harry Potter,Harry Potter and the Prisoner of Azkaban,4.56,1999
3,Harry Potter,Harry Potter and the Goblet of Fire,4.55,2000
4,Harry Potter,Harry Potter and the Order of the Phoenix,4.49,2003
5,Harry Potter,Harry Potter and the Half-Blood Prince,4.57,2005
6,Harry Potter,Harry Potter and the Deathly Hallows,4.61,2007
7,The Lord of the Rings,The Fellowship of the Ring,4.36,1954
8,The Lord of the Rings,The Two Towers,4.44,1954
9,The Lord of the Rings,The Return of the King,4.53,1955


We want to summarise by series, the `.groupby` method gives you the name of the column which has the groups

The result is a Python object which we will use for the next step

In [ ]:
volumes_notitle = volumes.drop(columns=['Title']) # Remove title, otherwise warning in next step!
groups = volumes_notitle.groupby('Series')
groups

We will use `count` and `mean` to work out the number of books and the average rating, and use `min` to work out the first publication year.

Other operations available include `sum` and `max` (you can check others [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html))

Now we have our data summary by groups

In [ ]:
summary = groups.agg({'count', 'mean', 'min'})
summary

Rating                  Year                   
                         min count      mean   min count         mean
Series                                                               
Game of Thrones         4.13     5  4.372000  1996     5  2002.000000
Harry Potter            4.42     7  4.524286  1997     7  2001.285714
The Hunger Games        4.03     3  4.216667  2008     3  2009.000000
The Lord of the Rings   4.36     3  4.443333  1954     3  1954.333333

### Joining Aggregated Data

We now have a dataframe with two sub-frames (one for `Rating` and one for `Year`) we can easily separate them

In [ ]:
rating_summary = summary['Rating']
rating_summary

,min,count,mean
Series,,,
Game of Thrones,4.13,5,4.372000
Harry Potter,4.42,7,4.524286
The Hunger Games,4.03,3,4.216667
The Lord of the Rings,4.36,3,4.443333


In [ ]:
year_summary = summary['Year']
year_summary

,min,count,mean
Series,,,
Game of Thrones,1996,5,2002.000000
Harry Potter,1997,7,2001.285714
The Hunger Games,2008,3,2009.000000
The Lord of the Rings,1954,3,1954.333333


Let's rename the columns for the `Ratings` aggregation, and remove the unneeded `min` column

In [ ]:
rating_summary = summary['Rating'].rename(
    columns={'mean' : 'Average Rating',
             'count' : 'Number of Books'})
rating_summary = rating_summary.drop(columns={'min'})
rating_summary

,Number of Books,Average Rating
Series,,
Game of Thrones,5,4.372000
Harry Potter,7,4.524286
The Hunger Games,3,4.216667
The Lord of the Rings,3,4.443333


We may also want to round off the average ratings

In [ ]:
rating_summary['Average Rating'] = rating_summary['Average Rating'].round(2)

rating_summary

,Number of Books,Average Rating
Series,,
Game of Thrones,5,4.37
Harry Potter,7,4.52
The Hunger Games,3,4.22
The Lord of the Rings,3,4.44


Let's also rename the column from the `Year` aggregation and drop the other columns

In [ ]:
year_summary = year_summary.rename(columns={'min' : 'First Published'})
year_summary = year_summary.drop(columns={'mean', 'count'})
year_summary

,First Published
Series,
Game of Thrones,1996
Harry Potter,1997
The Hunger Games,2008
The Lord of the Rings,1954


Now we can join the `books`, `rating_summary` and `year_summary` dataframes

Since all have the same keys we don't need to worry about join type, but this is a left join so will keep everything in the `books` data frame if it didn't match

In [ ]:
books.join(rating_summary).join(year_summary)

,Author,Number of Books,Average Rating,First Published
Book Series Title,,,,
The Lord of the Rings,J. R. R. Tolkien,3,4.44,1954
Game of Thrones,George R. R. Martin,5,4.37,1996
Harry Potter,J. K. Rowling,7,4.52,1997
The Hunger Games,Suzanne Collins,3,4.22,2008


## How are aggregations "done" in Tableau?

Simply put, every time you drag and drop the variables, you choose your aggregation!

## Updating data frames

Let's create two example datasets, where the goal is to update information about supermarket products:

In [ ]:
products_2024 = pd.read_csv(
    'https://www.dropbox.com/scl/fi/osi4arzr15z8zts79ekxp/products_2024.csv?rlkey=zthvjx7rhhnes75s71kns1j0b&raw=1'
    , index_col=0)
products_2024

,Name,% Fat,Price per kg
ID,,,
3,Beef,10,30.00
76,Chicken,5,8.99
495,Lamb,25,40.00


In [ ]:
products_2025 = pd.read_csv(
    'https://www.dropbox.com/scl/fi/dhtcfrvwlv9beo9zrqwha/products_2025.csv?rlkey=f2gje1ukkiqfbxi38bcq1of79&raw=1'
    , index_col=0)
products_2025

,Name,Price per kg,GHGE per kg
ID,,,
76,Poultry,13.00,300
495,Lamb,45.00,1000
5176,Fish,11.49,120


Notice that:
* The name of product `ID=76` has changed
* There's a new product (`Fish`)
* One "old" product (`Beef`) wasn't updated at all
* `Price per kg` have been updated
* New data has been captured (`GHGE per kg`)
* Some "old" data hasn't been updated (`% FAT`)

How can we create an "updated" version of the data?

In [ ]:
# A simple join won't work!
products_2024.join(products_2025)

ValueError: columns overlap but no suffix specified: Index(['Name', 'Price per kg'], dtype='object')

In [ ]:
import numpy as np
# 1. Add a GHGE column to the 2024 data filled with NaN
products_2024["GHGE per kg"] = np.nan
# 2. Update the 2024 dataframe with the 2025 one
products_2024.update(products_2025)
# 3, Find the new entries
to_be_added = products_2025.loc[products_2025.index.difference(products_2024.index)]
# 4. Concatenate
products = pd.concat([products_2024, to_be_added])
products

,Name,% Fat,Price per kg,GHGE per kg
ID,,,,
3,Beef,10.0,30.00,NaN
76,Poultry,5.0,13.00,300.0
495,Lamb,25.0,45.00,1000.0
5176,Fish,NaN,11.49,120.0


## Melt and Pivoting

### Wide vs Long Data (a real-life example)

The pizza example; when orders are collected we create long data, but the kitchen needs wide data to manage it better!

Wide data contains a column for each variable, and a row for each entity

The "entity" ID (in this case `Name`, but it could be an ID, a number, etc.) is in the first column, or it could be the index

| Name    | Age  | Height | Hair Colour |
| ------: | ---: | -----: | :---------- |
| Alice   |   36 |  1.68  | Blonde      |
| Bob     |   28 |  1.73  | Red         |
| Charlie |   29 |  1.60  | -           |

Long data contains a row for each observation of a variable

This is also called entity-attribute-value data

Note that rows can be omitted if there is missing data

| Entity ID | Attribute / Variable   |   Value |
| --------: | :--------------------- | ------: |
| Alice     | Age                    |      36 |
| Bob       | Age                    |      28 |
| Charlie   | Age                    |      29 |
| Alice     | Height                 |    1.68 |
| Bob       | Height                 |    1.73 |
| Charlie   | Height                 |    1.60 |
| Alice     | Hair Colour            |  Blonde |
| Bob       | Hair Colour            |     Red |

USUALLY, long data has more rows than cols (and vice versa) but this is not always the case!

### Melting Wide to Long Data
![Fig. 1](https://www.dropbox.com/scl/fi/ys9b0bc83kzbo7mwqonab/fig1.jpg?rlkey=9y6opgfdwlpidau98pgf8kjyf&raw=1)

Once again, let's import a new dataset

In [ ]:
stock = pd.read_csv(
    'https://www.dropbox.com/scl/fi/dzgt50lp1uiw2x55kc94w/stock.csv?rlkey=ukal2d1hs4jaqs4jul3ah7f3u&raw=1')
stock

,Company,Symbol,1980,1990,2000,2010,2020
0,Apple,AAPL,0.51,1.22,3.88,37.53,318.73
1,Google,GOOGL,NaN,NaN,NaN,312.54,1518.73
2,Microsoft,MSFT,NaN,1.03,53.31,28.21,185.38


**What kind of data is this?**

**Which are the entities/ID columns?**

**Which are the attributes (variables) in this case?**

The entity/ID can be either `Company` or `Symbol`. The attribute is the price in different years.

Let's melt this data using the `.melt` method

In [ ]:
stock.melt(id_vars = ['Company', 'Symbol'])

,Company,Symbol,variable,value
0,Apple,AAPL,1980,0.51
1,Google,GOOGL,1980,NaN
2,Microsoft,MSFT,1980,NaN
3,Apple,AAPL,1990,1.22
4,Google,GOOGL,1990,NaN
5,Microsoft,MSFT,1990,1.03
6,Apple,AAPL,2000,3.88
7,Google,GOOGL,2000,NaN
8,Microsoft,MSFT,2000,53.31
9,Apple,AAPL,2010,37.53


We can ensure that the new columns get named correctly by using the `var_name` and `value_name` options

In [ ]:
stock = stock.melt(id_vars = ['Company', 'Symbol'], var_name='Year', value_name='Price (USD)')
stock

,Company,Symbol,Year,Price (USD)
0,Apple,AAPL,1980,0.51
1,Google,GOOGL,1980,NaN
2,Microsoft,MSFT,1980,NaN
3,Apple,AAPL,1990,1.22
4,Google,GOOGL,1990,NaN
5,Microsoft,MSFT,1990,1.03
6,Apple,AAPL,2000,3.88
7,Google,GOOGL,2000,NaN
8,Microsoft,MSFT,2000,53.31
9,Apple,AAPL,2010,37.53


The `NaN` values aren't really contributing anything and are just there because they were in the wide dataset, so let's remove them!

In [ ]:
stock = stock[stock['Price (USD)'].notnull()]
stock

,Company,Symbol,Year,Price (USD)
0,Apple,AAPL,1980,0.51
3,Apple,AAPL,1990,1.22
5,Microsoft,MSFT,1990,1.03
6,Apple,AAPL,2000,3.88
8,Microsoft,MSFT,2000,53.31
9,Apple,AAPL,2010,37.53
10,Google,GOOGL,2010,312.54
11,Microsoft,MSFT,2010,28.21
12,Apple,AAPL,2020,318.73
13,Google,GOOGL,2020,1518.73


If you prefer the data sorted by entity, then variable, you can re-sort

In [ ]:
stock.sort_values(['Symbol', 'Year'])

,Company,Symbol,Year,Price (USD)
0,Apple,AAPL,1980,0.51
3,Apple,AAPL,1990,1.22
6,Apple,AAPL,2000,3.88
9,Apple,AAPL,2010,37.53
12,Apple,AAPL,2020,318.73
10,Google,GOOGL,2010,312.54
13,Google,GOOGL,2020,1518.73
5,Microsoft,MSFT,1990,1.03
8,Microsoft,MSFT,2000,53.31
11,Microsoft,MSFT,2010,28.21


Once again, Python has not detected that the columns were integer types

This is because the years were obtained from the headers, and by default Python believes that these are strings

In [ ]:
stock[stock['Year'] >= 2000]  # Error!

TypeError: '>=' not supported between instances of 'str' and 'int'

As you can see, the data type is `object`, meaning these numbers are stored as `str`

In [ ]:
stock['Year']

,Year
0,1980
3,1990
5,1990
6,2000
8,2000
9,2010
10,2010
11,2010
12,2020
13,2020


The columns which was originally headings has become `object` (`str`) types

We can change the data type of the `Year` column using `astype`

In [ ]:
stock = stock.astype({'Year' : 'int64'})
stock

,Company,Symbol,Year,Price (USD)
0,Apple,AAPL,1980,0.51
3,Apple,AAPL,1990,1.22
5,Microsoft,MSFT,1990,1.03
6,Apple,AAPL,2000,3.88
8,Microsoft,MSFT,2000,53.31
9,Apple,AAPL,2010,37.53
10,Google,GOOGL,2010,312.54
11,Microsoft,MSFT,2010,28.21
12,Apple,AAPL,2020,318.73
13,Google,GOOGL,2020,1518.73


Now we are able to do numerical comparisons on the `Year` column

In [ ]:
stock[stock['Year'] >= 2000]

,Company,Symbol,Year,Price (USD)
6,Apple,AAPL,2000,3.88
8,Microsoft,MSFT,2000,53.31
9,Apple,AAPL,2010,37.53
10,Google,GOOGL,2010,312.54
11,Microsoft,MSFT,2010,28.21
12,Apple,AAPL,2020,318.73
13,Google,GOOGL,2020,1518.73
14,Microsoft,MSFT,2020,185.38


### Pivoting Long to Wide Data
![Fig. 2](https://www.dropbox.com/scl/fi/rib5iwn2awsz7sq3ak0yi/fig2.gif?rlkey=dmxm6kjzwfl58po25k6yef747&raw=1)

A new dataset...

In [ ]:
weather = pd.read_csv(
    'https://www.dropbox.com/scl/fi/wsufi0yxy3alenes77l4g/weather-canada.csv?rlkey=6yi3uyvm6lk3zm6y2z9tto8m1&raw=1')
weather

,Station Name,Province,Year,Mean Temperature (C),Total Precipitation (mm)
0,BEAR CREEK,BC,1971,15.4,20.9
1,COWICHAN BAY CHERRY POINT,BC,1971,17.4,12.8
2,COWICHAN LAKE FORESTRY,BC,1971,18.8,21.3
3,COWICHAN LAKE VILLAGE,BC,1971,17.7,36.4
4,DUNCAN FORESTRY,BC,1971,17.7,18.1
...,...,...,...,...,...
81757,GOOSE A,NL,2017,15.8,109.0
81758,HOPEDALE (AUT),NL,2017,11.6,83.2
81759,MARY'S HARBOUR A,NL,2017,14.5,56.9
81760,NAIN,NL,2017,10.6,38.3


This was long data, however we have multiple variables per observation

We can pivot the table to see each station with the entry corresponding to different years

In [ ]:
weather_p = weather.pivot(index='Station Name',
                          columns='Year',
                          values=['Mean Temperature (C)',
                                  'Total Precipitation (mm)'])
weather_p

Mean Temperature (C)                                \
Year                                     1971  1972  1973  1974  1975  1976   
Station Name                                                                  
(AE) BOW SUMMIT                           NaN   NaN   NaN   NaN   NaN   NaN   
100 MILE HOUSE                           16.0  15.1  15.1  13.7  17.2  14.5   
100 MILE HOUSE 6NE                        NaN   NaN   NaN   NaN   NaN   NaN   
108 MILE HOUSE                            NaN   NaN  15.9   NaN   NaN   NaN   
108 MILE HOUSE ABEL LAKE                  NaN   NaN   NaN   NaN   NaN   NaN   
...                                       ...   ...   ...   ...   ...   ...   
YOYO                                      NaN   NaN   NaN   NaN   NaN   NaN   
ZAMA LO                                  15.7  14.3  15.3  13.5  17.5  14.3   
ZEBALLOS MURAUDE CREEK                    NaN   NaN   NaN   NaN   NaN   NaN   
ZEHNER                                    NaN   NaN   NaN   NaN   NaN   NaN   
ZHODA                                     NaN   NaN   NaN   NaN   NaN   NaN   

                                                  ...  \
Year                      1977  1978  1979  1980  ...   
Station Name                                      ...   
(AE) BOW SUMMIT            NaN   NaN   NaN   NaN  ...   
100 MILE HOUSE            13.5  15.5  16.0  14.1  ...   
100 MILE HOUSE 6NE         NaN   NaN   NaN   NaN  ...   
108 MILE HOUSE             NaN   NaN   NaN   NaN  ...   
108 MILE HOUSE ABEL LAKE   NaN   NaN   NaN   NaN  ...   
...                        ...   ...   ...   ...  ...   
YOYO                       NaN   NaN   NaN   NaN  ...   
ZAMA LO                   13.6  14.6  18.1  16.0  ...   
ZEBALLOS MURAUDE CREEK     NaN   NaN   NaN   NaN  ...   
ZEHNER                     NaN   NaN   NaN   NaN  ...   
ZHODA                      NaN   NaN   NaN   NaN  ...   

                         Total Precipitation (mm)                           \
Year                                         2008  2009  2010   2011  2012   
Station Name                                                                 
(AE) BOW SUMMIT                               NaN   NaN   NaN    NaN   NaN   
100 MILE HOUSE                                NaN   NaN   NaN    NaN   NaN   
100 MILE HOUSE 6NE                           48.4  47.0  21.4   75.8  68.4   
108 MILE HOUSE                                NaN   NaN   NaN    NaN   NaN   
108 MILE HOUSE ABEL LAKE                     37.6  42.8  12.0   55.4  57.2   
...                                           ...   ...   ...    ...   ...   
YOYO                                          NaN   NaN   NaN    NaN   NaN   
ZAMA LO                                      81.2  42.4  62.8  175.0   NaN   
ZEBALLOS MURAUDE CREEK                        NaN   NaN   NaN  123.2  56.4   
ZEHNER                                        NaN   NaN   NaN    NaN   NaN   
ZHODA                                         NaN   NaN   NaN    NaN   NaN   

                                                          
Year                      2013   2014  2015   2016  2017  
Station Name                                              
(AE) BOW SUMMIT            NaN    NaN   NaN    NaN   NaN  
100 MILE HOUSE             NaN    NaN   NaN    NaN   NaN  
100 MILE HOUSE 6NE        28.8  112.4  93.2  106.4   4.8  
108 MILE HOUSE             NaN    NaN   NaN    NaN   NaN  
108 MILE HOUSE ABEL LAKE  19.9   30.4  51.8   37.6   3.2  
...                        ...    ...   ...    ...   ...  
YOYO                       NaN    NaN   NaN    NaN   NaN  
ZAMA LO                    NaN    NaN   NaN    NaN   NaN  
ZEBALLOS MURAUDE CREEK     2.4   75.9  62.8   89.4  47.9  
ZEHNER                     NaN    NaN   NaN    NaN   NaN  
ZHODA                      NaN    NaN   NaN    NaN   NaN  

[4808 rows x 94 columns]

Even after the pivot, there are more rows than cols

You can see that the method was capable of "grouping" data, either for mean temp or precipitations!

This allows us to create new dataframes based only on the required info

In [ ]:
temperature = weather_p['Mean Temperature (C)']
temperature

Year,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Station Name,,,,,,,,,,,,,,,,,,,,,
(AE) BOW SUMMIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100 MILE HOUSE,16.0,15.1,15.1,13.7,17.2,14.5,13.5,15.5,16.0,14.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100 MILE HOUSE 6NE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.8,16.5,14.6,11.2,15.1,14.5,16.6,16.5,14.9,17.3
108 MILE HOUSE,NaN,NaN,15.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108 MILE HOUSE ABEL LAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.1,18.7,16.8,14.4,18.0,17.5,18.5,18.0,16.7,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOYO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZAMA LO,15.7,14.3,15.3,13.5,17.5,14.3,13.6,14.6,18.1,16.0,...,14.9,11.1,9.5,13.5,NaN,NaN,NaN,NaN,NaN,NaN
ZEBALLOS MURAUDE CREEK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,13.9,16.4,18.4,17.6,19.3,17.3,16.3


In [ ]:
temp2010 = temperature[[2010]]
temp2010 = temp2010[temp2010[2010].notnull()]
temp2010

Year,2010
Station Name,
100 MILE HOUSE 6NE,14.6
108 MILE HOUSE ABEL LAKE,16.8
ABBOTSFORD A,18.3
ABEE AGDM,15.3
ACADIA VALLEY,17.8
...,...
YOHIN,17.7
YOHO NP OHARA LAKE,10.0
YOHO PARK,12.0


In [ ]:
temp2010.mean()

,0
Year,
2010,17.429932


Let's pivot again, this time with the `Year` as index

In [ ]:
weather_p2 = weather.pivot(index='Year',
                           columns='Station Name',
                           values=['Mean Temperature (C)',
                                   'Total Precipitation (mm)'])

weather_p2

Mean Temperature (C)                                    \
Station Name      (AE) BOW SUMMIT 100 MILE HOUSE 100 MILE HOUSE 6NE   
Year                                                                  
1971                          NaN           16.0                NaN   
1972                          NaN           15.1                NaN   
1973                          NaN           15.1                NaN   
1974                          NaN           13.7                NaN   
1975                          NaN           17.2                NaN   
1976                          NaN           14.5                NaN   
1977                          NaN           13.5                NaN   
1978                          NaN           15.5                NaN   
1979                          NaN           16.0                NaN   
1980                          NaN           14.1                NaN   
1981                          NaN           15.1                NaN   
1982                          NaN           14.8                NaN   
1983                          NaN           14.4                NaN   
1984                          NaN           14.7                NaN   
1985                          NaN           16.6                NaN   
1986                          NaN           13.5                NaN   
1987                          NaN           15.4               15.1   
1988                          NaN           14.9               14.7   
1989                          NaN           15.0               14.8   
1990                          NaN           16.3               16.0   
1991                          NaN           14.9               14.5   
1992                          NaN           15.7               15.3   
1993                          NaN           13.5               13.3   
1994                          NaN           16.7               16.6   
1995                          NaN           15.9               15.5   
1996                          NaN           15.3               15.4   
1997                          NaN           14.9               14.5   
1998                         12.1           18.1               17.4   
1999                          NaN           14.9               14.1   
2000                          9.1            NaN               14.6   
2001                          9.6            NaN               14.3   
2002                         10.0            NaN               15.2   
2003                         11.1            NaN               16.2   
2004                          NaN            NaN                NaN   
2005                          NaN            NaN               14.4   
2006                          NaN            NaN               16.8   
2007                          NaN            NaN               16.6   
2008                          NaN            NaN               14.8   
2009                          NaN            NaN               16.5   
2010                          NaN            NaN               14.6   
2011                          NaN            NaN               11.2   
2012                          NaN            NaN               15.1   
2013                          NaN            NaN               14.5   
2014                          NaN            NaN               16.6   
2015                          NaN            NaN               16.5   
2016                          NaN            NaN               14.9   
2017                          NaN            NaN               17.3   

                                                                        \
Station Name 108 MILE HOUSE 108 MILE HOUSE ABEL LAKE 150 MILE HOUSE 7N   
Year                                                                     
1971                    NaN                      NaN              15.5   
1972                    NaN                      NaN              14.1   
1973                   15.9                      NaN              14.1   
1974              

As you can see, it all depends on what we consider the entity to be

This allows us to get the mean of **all** stations!

In [ ]:
weather_p2['Mean Temperature (C)'].mean()

,0
Station Name,
(AE) BOW SUMMIT,10.380000
100 MILE HOUSE,15.217241
100 MILE HOUSE 6NE,15.243333
108 MILE HOUSE,15.900000
108 MILE HOUSE ABEL LAKE,15.883333
...,...
YOYO,13.350000
ZAMA LO,15.392500
ZEBALLOS MURAUDE CREEK,17.028571


### Pivoting with Aggregation

This next cell will give an error! **WHY**

In [ ]:
weather.pivot(index='Province',
              columns='Year',
              values=['Mean Temperature (C)',
                    'Total Precipitation (mm)'])  # Error!

ValueError: Index contains duplicate entries, cannot reshape

The index cannot be a duplicated value (i.e. there are several stations in each province)

If we use `pivot_table` with `aggfunc`, we can tell Pandas what to do with these values for instance we may want the `mean`

In [ ]:
weather.pivot_table(index='Province',
                    columns='Year',
                    values=['Mean Temperature (C)', 'Total Precipitation (mm)'],
                    aggfunc='mean')

Mean Temperature (C)                                              \
Year                     1971       1972       1973       1974       1975   
Province                                                                    
AB                  15.057655  13.122741  15.160550  14.393491  17.158160   
BC                  17.103300  15.994702  15.956347  14.695879  17.289267   
MB                  16.250000  16.147368  17.946281  20.249194  20.560504   
NB                  18.055172  17.520968  19.972581  17.357143  19.917742   
NL                  14.676596  14.482456  16.917241  13.165000  17.165517   
NS                  18.052174  17.608451  19.455844  16.082895  19.432877   
NT                  13.304545  11.672727  14.507143  13.164286  14.565789   
NU                   6.747222   4.506452   6.985294   7.532432   6.761765   
ON                  18.109699  18.820209  19.890492  19.444156  20.575974   
PE                  18.700000  18.006667  20.486667  17.133333  20.940000   
QC                  17.190986  17.606268  19.279911  17.842729  19.930769   
SK                  16.155233  15.250568  17.767895  19.021081  20.272826   
YT                  13.860000  12.914286  12.732258  11.845714  13.639474   

                                                                 ...  \
Year           1976       1977       1978       1979       1980  ...   
Province                                                         ...   
AB        15.045833  13.898171  15.229518  16.707165  15.106563  ...   
BC        14.965147  15.166482  17.168539  17.047714  15.647929  ...   
MB        18.756303  18.335897  17.756364  19.988288  18.857273  ...   
NB        17.650820  18.026230  18.147368  19.105357  17.536842  ...   
NL        14.661818  14.688235  15.235088  15.448276  13.847692  ...   
NS        17.419737  17.687500  17.256164  18.333803  17.115714  ...   
NT        13.507692  12.472000  11.266667  15.052000  12.096000  ...   
NU         6.460000   6.934375   4.663636   6.081818   6.170000  ...   
ON        18.717377  19.830201  19.069180  19.676431  19.504667  ...   
PE        18.200000  18.257143  18.371429  19.078571  17.371429  ...   
QC        17.253037  17.449657  17.867352  18.817401  17.412963  ...   
SK        18.302260  17.582065  17.468306  19.556425  18.334637  ...   
YT        12.808333  12.702941  13.840000  13.700000  13.000000  ...   

         Total Precipitation (mm)                                      \
Year                         2008        2009        2010        2011   
Province                                                                
AB                      57.499465   71.784946   63.541739   90.536782   
BC                      45.344048   33.367480   16.013248   70.216800   
MB                      91.120000   79.639394   76.880952   57.208197   
NB                      90.202941  150.363636  100.396552  126.335714   
NL                      66.025397  104.258333  122.348000  138.456522   
NS                      83.421429   93.652500   96.146875  103.189189   
NT                      29.204878   43.634483   44.984211   43.134483   
NU                      33.063889   29.728571   33.527027   24.541379   
ON                     103.389080   96.332143   88.679268   63.204516   
PE                      54.588889  139.633333  123.988889  121.822222   
QC                     129.135885  138.533659   87.888780   93.660101   
SK                      76.864211   73.309574   71.457143   74.325843   
YT                      62.766667   24.854167   48.369231   64.056000   

                                                                               
Year           2012        2013        2014        2015       2016       2017  
Province                                                                       
AB        72.519481   54.610132   46.419178   55.166038  77.486829  45.088500  
BC        46.386822   19.295000   43.874000   38.252863  50.727064  24.443781  
MB        61.190625   77.395238   43.877778   98.64310

You can also set different aggregation functions for each variable:

In [ ]:
weather.pivot_table(index='Province',
                    columns='Year',
                    values=['Mean Temperature (C)', 'Total Precipitation (mm)'],
                    aggfunc=['mean','sum'])

mean                                              \
         Mean Temperature (C)                                               
Year                     1971       1972       1973       1974       1975   
Province                                                                    
AB                  15.057655  13.122741  15.160550  14.393491  17.158160   
BC                  17.103300  15.994702  15.956347  14.695879  17.289267   
MB                  16.250000  16.147368  17.946281  20.249194  20.560504   
NB                  18.055172  17.520968  19.972581  17.357143  19.917742   
NL                  14.676596  14.482456  16.917241  13.165000  17.165517   
NS                  18.052174  17.608451  19.455844  16.082895  19.432877   
NT                  13.304545  11.672727  14.507143  13.164286  14.565789   
NU                   6.747222   4.506452   6.985294   7.532432   6.761765   
ON                  18.109699  18.820209  19.890492  19.444156  20.575974   
PE                  18.700000  18.006667  20.486667  17.133333  20.940000   
QC                  17.190986  17.606268  19.279911  17.842729  19.930769   
SK                  16.155233  15.250568  17.767895  19.021081  20.272826   
YT                  13.860000  12.914286  12.732258  11.845714  13.639474   

                                                                 ...  \
                                                                 ...   
Year           1976       1977       1978       1979       1980  ...   
Province                                                         ...   
AB        15.045833  13.898171  15.229518  16.707165  15.106563  ...   
BC        14.965147  15.166482  17.168539  17.047714  15.647929  ...   
MB        18.756303  18.335897  17.756364  19.988288  18.857273  ...   
NB        17.650820  18.026230  18.147368  19.105357  17.536842  ...   
NL        14.661818  14.688235  15.235088  15.448276  13.847692  ...   
NS        17.419737  17.687500  17.256164  18.333803  17.115714  ...   
NT        13.507692  12.472000  11.266667  15.052000  12.096000  ...   
NU         6.460000   6.934375   4.663636   6.081818   6.170000  ...   
ON        18.717377  19.830201  19.069180  19.676431  19.504667  ...   
PE        18.200000  18.257143  18.371429  19.078571  17.371429  ...   
QC        17.253037  17.449657  17.867352  18.817401  17.412963  ...   
SK        18.302260  17.582065  17.468306  19.556425  18.334637  ...   
YT        12.808333  12.702941  13.840000  13.700000  13.000000  ...   

                              sum                                      \
         Total Precipitation (mm)                                       
Year                         2008     2009     2010     2011     2012   
Province                                                                
AB                        21504.8  26704.0  21921.9  31506.8  16752.0   
BC                        11426.7   8208.4   3747.1  17554.2  11967.8   
MB                         6834.0   5256.2   4843.5   3489.7   3916.2   
NB                         3066.9   4962.0   2911.5   3537.4   1254.3   
NL                         4159.6   6255.5   6117.4   6369.0   4653.1   
NS                         3503.7   3746.1   3076.7   3818.0   2341.6   
NT                         1197.4   1265.4   1709.4   1250.9   1025.8   
NU                         1190.3   1040.5   1240.5    711.7   1393.9   
ON                        17989.7  16183.8  14543.4   9796.7   9757.9   
PE                          491.3   1256.7   1115.9   1096.4    487.0   
QC                        26989.4  28399.4  18017.2  18544.7  12183.2   
SK                         7302.1   6891.1   6502.6   6615.0   5312.2   
YT                         1318.1    596.5   1257.6   1601.4   1219.4   

                                                       
                                                       
Year         2013     2014     2015     2016     2017  
Province                                               
AB        12396.5  10165

Now, let's **leverage** the power of AI to get better explanations of the errors obtained before!

## Lab